# Memory

- LLM과 주고 받은 대화를 저장하는 기능을 말한다. 
  - LLM 모델은 대화의 상태를 저장하지 않는다. 그래서 질문을 하면 그것에 대한 답변을 하고 끝이다. 대화 내용에 따라 이전 대화 내용을 바탕으로 연결되는 질문을 하고 그것에 대한 답변을 받아야 할 때가 있다. 이런 경우 지금 까지의 대화 내용을 저장하는 것을 메모리(memory)라고 한다.
- **방식**
  - 대화 내용을 저장한 뒤 다음 질문을 할 때 저장된 이전 질문들을 합쳐서 전송한다.
  - 이전 대화내용을 어떻게 저장하는지에 따라 다양한 방식의 memory 기능이 있다.
    - LLM은 [입력 토큰수의 제한이](https://platform.openai.com/docs/models) 있기 때문에 대화를 무한정 저장할 수없다.
    - Langchain은 이전 대화 내용들을 요약하거나 최신 몇개만 저장하는 방식의 다양한 memory 방식을 제공한다.

![memory.png](figures/memory.png)

## 주요 Memory Class
### ConversationBufferMemory
- https://python.langchain.com/v0.1/docs/modules/memory/types/buffer/
- **대화를 모두 그대로 저장**한다.
- 대화가 길어질 경우 저장 양이 많아지는 문제가 있다.

### ConversationBufferWindowMemory
- **최신 대화 K개만 저장** 하고 그 이전 대화는 삭제한다. (K를 window size라고 하고 객체 생성시 설정한다.)
  - 한개의 대화는 input(질문)-output(답변) 한 쌍을 말한다.
    
### ConversationTokenBufferMemory
- https://python.langchain.com/v0.1/docs/modules/memory/types/token_buffer/
- 지정한 **token 수를 넘지 않는 범위**에서 최신 대화들을 저장한다.
  

### ConversationSummaryMemory
- https://python.langchain.com/v0.1/docs/modules/memory/types/summary/
- **기존 대화들을 요약**해서 저장한다. 다만 요약은 llm이 하기 때문에 객체 생성시 llm 모델을 지정해야 한다.
- 대화 내용이 요약되어 전송되므로 토큰 수를 줄여 요금을 절약할 수있다.
  
### ConversationSummaryBufferMemory
- https://python.langchain.com/v0.1/docs/modules/memory/types/summary_buffer/
- 대화들 자체를 메모리에 저장하다가 지정한 token을 넘어가면 **오래된 대화들 순서대로 요약**한다. 
  - 최신대화는 그대로 저장하고 오래된 대화는 요약해서 저장하는 방식.

> #### ConversationSummaryMemory 토큰 비용
> - 초기 비용은 ConversationSummaryMemory가 기존 대화를 요약하기 위해 LLM을 사용하므로 ConversationBufferMemory보다 더 많은 토큰을 사용한다. 그래서 비용이 더 많이 든다.
> - 대화가 길어질수록 ConversationBufferMemory모든 대화 내용을 그대로 저장하므로 토큰 수가 선형적으로 증가한다. 
> - 반면 ConversationSummaryMemory는 대화가 길어질수록 요약된 형태로 기존 대화들이 저장되어 토큰 증가율이 더 낮아지게 되서 비용을 절감할 수있다.
> - 비용과 관련해 **짧은 대화**일 경우 ConversationBufferMemory 가 효율적이고 **긴 대화일 경우** ConversationSummaryMemory가 효율적이다.
> - **요약 비용을 절감** 하기 위해 **요약에는 저렴한 모델**을 사용할 수 있다.

> #### Deprecated
> - 위 메모리 저장 방식은 0.3.1 부터 deprecated 되었다. (1.0 버전에서 제거될 예정) 
> - 대신 RunnableWithMessageHistory 사용이 권장 된다.

### 공통 메소드
- **initializer**
  - memory_key: str = "history"
    - 대화내역을 dict로 저장하는데 그때 사용되는 key.
    - default는 **"history"**
    - Prompt template에서 memory의 대화내용을 저장하는 placeholder(template variable)의 이름을 이 memory_key 로 지정한다.
  - return_message: bool
    - True: 각 대화내용을 Message(HumanMessage, AIMessage) 객체에 저장하고 그것을 List로 묶어서 반환
    - False: 대화내용을 문자열(str)로 반환한다.
  - chat_memory: BaseChatMessageHistory
    - 대화 history를 어디에 저장할 지 설정. (메모리, sql등)
- **save_context(inputs: dict, outputs: dict)**
  - Memory에 대화내용(Context)을 저장.
  - 파라미터
    - inputs: Human message
    - outputs: AI message
- **load_memory_variables(dict)**
  - 저장된 대화내용(context)를 반환.
  - argument로 빈 dict를 넣어준다.
- **clear()**
  - 저장된 모든 대화 비우기


## 대화 저장

In [6]:
from langchain.memory import ConversationBufferMemory # 모든 대화를 다 저장.
from pprint import pprint
# 메모리 객체 생성
memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True, # 대화내용을 Message객체로 반환. False: 문자열
)

# 메모리에 대화 저장.
memory.save_context(
    inputs={"human":"안녕하세요. 은행 계좌를 개설 하고 싶습니다."}, #{role:대화내용}
    outputs={"ai":"계좌 개설을 원하시는 군요. 신분증을 먼저 준비하세요."}
)
# 저장된 대화들을 조회
save_conv = memory.load_memory_variables({})
pprint(save_conv)
# 저장형식: dictionary
### {memory_key: "대화들"}

{'chat_history': [HumanMessage(content='안녕하세요. 은행 계좌를 개설 하고 싶습니다.', additional_kwargs={}, response_metadata={}),
                  AIMessage(content='계좌 개설을 원하시는 군요. 신분증을 먼저 준비하세요.', additional_kwargs={}, response_metadata={})]}


In [7]:
# save_conv['history']
save_conv['chat_history']

[HumanMessage(content='안녕하세요. 은행 계좌를 개설 하고 싶습니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='계좌 개설을 원하시는 군요. 신분증을 먼저 준비하세요.', additional_kwargs={}, response_metadata={})]

In [8]:
memory.save_context(
    inputs={"human":"신분증을 준비했습니다. 다음에는 무엇을 하면 되나요?"},
    outputs={"ai":"신분증 앞면을 촬영해서 업로드 해주세요."}
)
pprint(memory.load_memory_variables({}))

{'chat_history': [HumanMessage(content='안녕하세요. 은행 계좌를 개설 하고 싶습니다.', additional_kwargs={}, response_metadata={}),
                  AIMessage(content='계좌 개설을 원하시는 군요. 신분증을 먼저 준비하세요.', additional_kwargs={}, response_metadata={}),
                  HumanMessage(content='신분증을 준비했습니다. 다음에는 무엇을 하면 되나요?', additional_kwargs={}, response_metadata={}),
                  AIMessage(content='신분증 앞면을 촬영해서 업로드 해주세요.', additional_kwargs={}, response_metadata={})]}


In [9]:
memory.clear() # 저장내용 모두 삭제
pprint(memory.load_memory_variables({}))

{'chat_history': []}


# 주요 메모리 클래스 예제

## ConversationBufferMemory
메모리에 대화를 모두 저장한다.

In [ ]:
from langchain.memory import ConversationBufferMemory
from pprint import pprint

memory = ConversationBufferMemory(
    return_messages=True
)

memory.save_context(
    inputs={
        "human":"안녕하세요"
    },
    outputs={
        "ai": "반갑습니다."
    }
)

memory.save_context(
    inputs={
        "human":"한국 여행에 대해 물어보려고 합니다."
    },
    outputs={
        "ai":"무엇이든 물어보세요."
    }
)

memory.save_context(
    inputs={
        "human":"여행 할 때 꼭 먹어봐야할 음식 3개만 추천해줘."
    },
    outputs={
        "ai":"불고기, 비빔밥, 삼겹살을 추천합니다."
    }
)

memory.save_context(
    inputs={
        "human":"여행지 두 곳을 추천해줘."
    },
    outputs={
        "ai":"경복궁과 국립중앙박물관입니다."
    }
)
memory.save_context(
    inputs={
        "human":"여행지 한 곳 더 추천해줘."
    },
    outputs={
        "ai":"민속촌을 추천합니다."
    }
)

memory.save_context(
    inputs={
        "human":"한국의 자연을 느낄 수있는 여행지를 알려줘."
    },
    outputs={
        "ai":"북한산을 추천합니다. 서울 도심에서 자연을 느낄 수있습니다."
    }
)

pprint(memory.load_memory_variables({}))

{'history': [HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
             AIMessage(content='반갑습니다.', additional_kwargs={}, response_metadata={}),
             HumanMessage(content='한국 여행에 대해 물어보려고 합니다.', additional_kwargs={}, response_metadata={}),
             AIMessage(content='무엇이든 물어보세요.', additional_kwargs={}, response_metadata={}),
             HumanMessage(content='여행 할 때 꼭 먹어봐야할 음식 3개만 추천해줘.', additional_kwargs={}, response_metadata={}),
             AIMessage(content='불고기, 비빔밥, 삼겹살을 추천합니다.', additional_kwargs={}, response_metadata={}),
             HumanMessage(content='여행지 두 곳을 추천해줘.', additional_kwargs={}, response_metadata={}),
             AIMessage(content='경복궁과 국립중앙박물관입니다.', additional_kwargs={}, response_metadata={}),
             HumanMessage(content='여행지 한 곳 더 추천해줘.', additional_kwargs={}, response_metadata={}),
             AIMessage(content='민속촌을 추천합니다.', additional_kwargs={}, response_metadata={}),
             HumanMessage(content='한국의 

In [16]:
# LLM 모델에 요청
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4o-mini")
prompt_template = ChatPromptTemplate(
    [
        ("system", "당신은 한국 여행 전문가 입니다. 한국 여행과 관련된 다양한 정보를 알려주세요."),
        MessagesPlaceholder("history", optional=True),
        ("human", "{query}")
    ]
)
chain = prompt_template | model | StrOutputParser()

In [21]:
query = "마지막 소개한 여행지를 가는 방법에 대해 알려주세요."
res = chain.invoke(
    {"query":query, "history":memory.load_memory_variables({})["history"]}
)

In [22]:
print(res)

북한산은 서울에 위치해 있어 접근성이 좋습니다. 다음은 북한산을 가는 방법입니다:

1. **지하철 이용**:
   - **경복궁역 (3호선)**에서 하차 후 3번 출구로 나옵니다. 
   - 그 후, 도보로 약 20분 정도 걸어서 북한산으로 향할 수 있습니다.
   
   또는

   - **구파발역 (6호선)**에서 하차 후 1번 출구로 나와 버스를 이용할 수 있습니다.

2. **버스 이용**:
   - 서울 시내에서 북한산으로 가는 여러 버스 노선이 있습니다. 예를 들어, 34번, 704번, 720번 등의 버스를 이용하여 북한산 입구에 도착할 수 있습니다.

3. **자동차 이용**:
   - 자가용이나 렌트카를 이용할 경우, 내비게이션에 "북한산 국립공원"을 입력하면 편리하게 이동할 수 있습니다. 주차장이 마련되어 있으나 주말에는 혼잡할 수 있으니 참고하세요.

북한산은 다양한 등산로와 아름다운 경치를 제공하니 꼭 즐기시길 바랍니다!


In [23]:
## 최신 대화를 memery에 저장
memory.save_context(
    inputs={"human":query},
    outputs={"ai":res}
)

In [24]:
pprint(memory.load_memory_variables({})["history"])

[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='반갑습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='한국 여행에 대해 물어보려고 합니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='무엇이든 물어보세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행 할 때 꼭 먹어봐야할 음식 3개만 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='불고기, 비빔밥, 삼겹살을 추천합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행지 두 곳을 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='경복궁과 국립중앙박물관입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행지 한 곳 더 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='민속촌을 추천합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='한국의 자연을 느낄 수있는 여행지를 알려줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='북한산을 추천합니다. 서울 도심에서 자연을 느낄 수있습니다.', addition

## ConversationBufferWindowMemory
최신 대화 K개만 저장 한다.

## ConversationTokenBufferMemory
대화의 토큰 길이를 기준으로 최신 대화만 저장한다.

## ConversationSummaryMemory
대화를 요약해서 저장한다.

## ConversationSummaryBufferMemory
오래된 대화는 요약하고 최신대화는 그대로 저장한다.

# Chain
## Off-the-shelf chains

## LECL